<a href="https://colab.research.google.com/github/ejmata2/python/blob/main/multiprocessing_en_Windows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Por qué el módulo multiprocessing no funciona igual en Linux que en Windows

https://medium.com/@grvsinghal/speed-up-your-python-code-using-multiprocessing-on-windows-and-jupyter-or-ipython-2714b49d6fac
https://stackoverflow.com/questions/20222534/python-multiprocessing-on-windows-if-name-main

In [ ]:
El siguiente código se ejecuta correctamente en Linux, pero no en Windows:

In [ ]:
# Importar Pool
import multiprocessing as mp

# Definir una función para ejecutarla en paralelo
def worker(x):
 return x*x

# Obtener el número de cores del sistema o definir un número concreto
# num_cores = mp.cpu_count()
num_cores = 8

#Crear un pool de procesos
p=mp.Pool(processes = num_cores)

# Iniciar el pool para que trabajen en paralelo
output = p.map(worker,[i for i in range(0,num_cores)])
print(output)

[0, 1, 4, 9, 16, 25, 36, 49]


Si lo ejecutamos en una shell de Python en Windows (cmd → python), se mostrará un error como éste: 
> Can't get attribute ‘worker' on <module ‘\_\_main\_\_' \(built-in\)> 

Y si lo ejecutamos en Jupiter/Windows, se queda atascado, nunca se completará la tarea y no se mostrará ningún mensaje.

Esta se puede leer en la documentación sobre el módulo [multiprocessing](https://docs.python.org/3/library/multiprocessing.html) de Python 3.

> Note: Functionality within this package requires that the \_\_main\_\_ module be importable by the children. This is covered in [Programming guidelines](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool) however it is worth pointing out here. This means that some examples, such as the [multiprocessing.pool.Pool](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool) examples will not work in the interactive interpreter...

Para solucionarlo, hay que hacer lo siguiente:

1. Definir el worker en un fichero .py separado e importarlo. 

2. Añadir esta cláusula antes de llamar al worker
> if \_\_name\_\_ == '\_\_main\_\_:'

Suponiendo que guardamos el código en el fichero workers.py, quedaría asío:

In [ ]:
!echo "def worker(x):" > workers.py

In [ ]:
!echo "    return x*x" >> workers.py

In [ ]:
import multiprocessing as mp
import workers

if __name__ ==  '__main__': 
   # Obtener el número de cores del sistema o definir un número concreto
   # num_cores = mp.cpu_count()
   num_cores = 8

   #Crear un pool de procesos
   p=mp.Pool(processes = num_cores)

   # Iniciar el pool para que trabajen en paralelo
   output = p.map(workers.worker,[i for i in range(0,num_cores)])
   print(output)

https://stackoverflow.com/questions/20222534/python-multiprocessing-on-windows-if-name-main

You do not have to call Process() from the "top level" of the module. It is perfectly fine to call Process from a class method.

The only caveat is that you can not allow Process() to be called if or when the module is imported.

Since Windows has no fork, the multiprocessing module starts a new Python process and imports the calling module. If Process() gets called upon import, then this sets off an infinite succession of new processes (or until your machine runs out of resources). This is the reason for hiding calls to Process() inside

if __name__ == "__main__"
since statements inside this if-statement will not get called upon import.